In [1]:
#data prefix for output data
yearmonth = "202405"

In [2]:
#import necessary packages
import platform
import os
import sys
import time
#for managing the data
import pandas as pd

#install !pip install webdriver-manager
#webdrivers
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup

import pathlib

import os
from io import StringIO

from tqdm import tqdm

In [3]:
#get current path
curr_path = os.getcwd()
project_tag = curr_path.split("/")[-1]
grouping = curr_path.split("/")[-2]
output_path = "../../../data/{}/{}/{}/".format(grouping,project_tag,yearmonth )

#create data output path if it doesn't exist
pathlib.Path(output_path).mkdir(parents=True, exist_ok=True) 

In [4]:
output_path

'../../../data/regional/wellington_duff_guel_health_unit/202405/'

In [5]:
#load in level 1 data
df = pd.read_csv(output_path + "1_level_table.csv")
print(df.shape)

(2708, 13)


In [6]:
#open web driver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [7]:
#place holder for final dataframe list
df_list = []

In [8]:


for i,r in tqdm( df.iterrows() ):
    #resume point in case it crashes
    last_i = len(df_list)-1
    if i<last_i:
        continue
    url = r['url_entity_partial']
    url_entity = r['url_entity_full']
    
    #load the main landing page
    driver.get(url_entity)
    
    try:
        #find disclaimer button
        disclamer_btn = driver.find_element(by='name', value="AcceptDisclaimerButton")
    
        #click the disclaimer button to move to the data page
        disclamer_btn.click()
    except:
        pass

    #place holder for extracted data
    data_url_list = []
    data_url_full_list = []
    
    #locate each table rows
    data_rows = driver.find_elements(By.CLASS_NAME, value="hovereffect")

    #if no inspection data skip to the next url
    if len(data_rows)==0:
        continue
    
    for row in data_rows:
        data_url_list.append( row.get_attribute("onclick") )
    
    url_prefix = "https://www.checkbeforeyouchoose.ca"
    for u in data_url_list:
        temp_url = url_prefix+u.split("'")[1]
        data_url_full_list.append(temp_url)
    
    tables = pd.read_html(StringIO((driver.page_source)))
    table  = tables[1]
    table['url_inspection'] = data_url_list
    table['url_inspection_full'] = data_url_full_list
    table['url_entity_full'] = url_entity

    df_list.append(table)
 

2708it [22:24,  2.01it/s]


In [9]:
final_df = pd.concat(df_list)
print(final_df.shape)

(26772, 8)


In [10]:
final_df

,Facility Name,Inspection Type,Critical Infractions Count,Non Critical Infractions Count,Inspection Date,url_inspection,url_inspection_full,url_entity_full
0,Seed - Guelph Community Health Centre (The),Required,0,0,22-Feb-2024,location.href = '/Inspection/Details/8d1f56e2-...,https://www.checkbeforeyouchoose.ca/Inspection...,https://www.checkbeforeyouchoose.ca/Facility/D...
0,10C Shared Space,Required,0,0,05-Apr-2024,location.href = '/Inspection/Details/489c641a-...,https://www.checkbeforeyouchoose.ca/Inspection...,https://www.checkbeforeyouchoose.ca/Facility/D...
1,10C Shared Space,Required,0,0,08-May-2023,location.href = '/Inspection/Details/f1785a11-...,https://www.checkbeforeyouchoose.ca/Inspection...,https://www.checkbeforeyouchoose.ca/Facility/D...
2,10C Shared Space,Required,0,0,17-Oct-2022,location.href = '/Inspection/Details/b9afabd4-...,https://www.checkbeforeyouchoose.ca/Inspection...,https://www.checkbeforeyouchoose.ca/Facility/D...
3,10C Shared Space,Required,0,0,05-Oct-2021,location.href = '/Inspection/Details/9149908c-...,https://www.checkbeforeyouchoose.ca/Inspection...,https://www.checkbeforeyouchoose.ca/Facility/D...
...,...,...,...,...,...,...,...,...
21,Win Chinese Food,Required,0,0,26-Oct-2018,location.href = '/Inspection/Details/4fe8c75a-...,https://www.checkbeforeyouchoose.ca/Inspection...,https://www.checkbeforeyouchoose.ca/Facility/D...
22,Win Chinese Food,Required,0,1,11-Jul-2018,location.href = '/Inspection/Details/ce9ec164-...,https://www.checkbeforeyouchoose.ca/Inspection...,https://www.checkbeforeyouchoose.ca/Facility/D...
23,Win Chinese Food,Required,0,0,22-Feb-2018,location.href = '/Inspection/Details/efd8ac1e-...,https://www.checkbeforeyouchoose.ca/Inspection...,https://www.checkbeforeyouchoose.ca/Facility/D...
24,Win Chinese Food,Required,0,0,03-Nov-2017,location.href = '/Inspection/Details/e5290b64-...,https://www.checkbeforeyouchoose.ca/Inspection...,https://www.checkbeforeyouchoose.ca/Facility/D...


In [11]:
final_df.to_csv( output_path + "2_level_table.csv", index=False)

In [12]:
#close webdriver
try:
    driver.close()
except:
    pass